# Description
In this project, i will read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data that's been provided(i had to hand-code some names, and needed to google to find out where teams are)!

For each sport I would answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. I calculated the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so, I sent in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. I found the average of the win/loss ratios for those cities which have multiple teams of a single sport. I only used data **from year 2018** for my analysis

# NOTES
1. I did not include data about the MLS or CFL in any of the work, because I was only interested in the Big 4 in this project.
2. I tackled the four correlation questions in the order NHL, NBA, MLB, NFL
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams.


In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
def my_cities():
    city =pd.read_html('assets/wikipedia_data.html')
    city = city[1]
    return city
    
my_cities = my_cities() # making my function a global variable...
my_cities

,Metropolitan area,Country,Pop.rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—
5,"Washington, D.C.",United States,7,6131977,4,Redskins,Nationals[note 10],Wizards[note 11],Capitals,5,D.C. United,—
6,Philadelphia,United States,8,6070500,4,Eagles,Phillies[note 12],76ers,Flyers[note 13],5,Union,—
7,Boston,United States,12,4794447,4,Patriots[note 14],Red Sox[note 15],Celtics,Bruins,5,Revolution,—
8,Minneapolis–Saint Paul,United States,17,3551036,4,Vikings,Twins,Timberwolves,Wild[note 16],5,United FC,—
9,Denver,United States,20,2853077,4,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18],5,Rapids,—


In [3]:
# cleaning the html file for easy referencing later on...

def citiees1()   :
    
    keep = [
        'Metropolitan area',  'Population (2016 est.)[8]',  'NFL', 'MLB', 'NBA', 'NHL'
    ]
    cities1 = my_cities[keep]
    cities1.columns = [city.strip() for city in cities1.columns]
    cities1.rename(
    columns = 
    {
        'Population (2016 est.)[8]' : 'Population',

    },
    inplace = True)
    
    cities1.replace(
    to_replace = '\[.+\]', value = '', regex = True, inplace = True) # removing square brackets from the elements
    
    cities1.drop(index = [51], inplace = True) # dropping the footer
    
    ## splitting the elements with multiple teams below...
    
    cities1['NHL'].replace(
    to_replace='RangersIslandersDevils', value= 'Rangers, Islanders, Devils', regex = True, inplace=True)
    
    cities1['NHL'].replace(
    to_replace='KingsDucks', value= 'Kings, Ducks', regex = True, inplace=True)

    cities1['NBA'].replace(
    to_replace='KnicksNets', value='Knicks, Nets', regex = True, inplace = True)
    
    cities1['NBA'].replace(
    to_replace='LakersClippers', value='Lakers, Clippers', regex = True, inplace = True)

    cities1['MLB'].replace(
    to_replace='YankeesMets', value ='Yankees, Mets', regex=True, inplace=True)
    
    cities1['MLB'].replace(
    to_replace='DodgersAngels', value ='Dodgers, Angels', regex=True, inplace=True)
    cities1['MLB'].replace(
    to_replace='GiantsAthletics', value ='Giants, Athletics', regex=True, inplace=True)
    cities1['MLB'].replace(
    to_replace='CubsWhite Sox', value ='Cubs, White Sox', regex=True, inplace=True)

    cities1['NFL'].replace(
    to_replace='GiantsJets', value='Giants, Jets', regex = True, inplace=True
    )
    cities1['NFL'].replace(
    to_replace='RamsChargers', value='Rams, Chargers', regex = True, inplace=True
    )
    cities1['NFL'].replace(
    to_replace='49ersRaiders', value='49ers, Raiders', regex = True, inplace=True
    )
    
    cities1['Metropolitan area'] = [towns.strip() for towns in cities1['Metropolitan area']] # stripping elements in metropolitan column
    cities1['Population'] = cities1['Population'].astype(int)
    return cities1
    
my_cities1 = citiees1() # making the function a global variable
my_cities1

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,"Giants, Jets","Yankees, Mets","Knicks, Nets","Rangers, Islanders, Devils"
1,Los Angeles,13310447,"Rams, Chargers","Dodgers, Angels","Lakers, Clippers","Kings, Ducks"
2,San Francisco Bay Area,6657982,"49ers, Raiders","Giants, Athletics",Warriors,Sharks
3,Chicago,9512999,Bears,"Cubs, White Sox",Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals,Wizards,Capitals
6,Philadelphia,6070500,Eagles,Phillies,76ers,Flyers
7,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild
9,Denver,2853077,Broncos,Rockies,Nuggets,Avalanche


In [4]:
## getting the hockey file, and cleaning it as below...

def hockey():
    hockey_data = pd.read_csv('assets/nhl.csv')
    
    hockey_data = hockey_data.iloc[:35, :] #selecting year 2018
    hockey_data.drop(index =[0, 9, 18, 26], inplace=True) # dropping headers
    
    hockey_data['team'].replace(
    to_replace='\*?', value= '', regex=True, inplace=True) # cleaning the elements in the 'team' column
    
    hockey_data['W'] = hockey_data['W'].astype(int) # converting data type from object to integer
    hockey_data['L'] = hockey_data['L'].astype(int) # converting data type from object to integer
    hockey_data['GP'] = hockey_data['GP'].astype(int) # converting data type from object to integer
    hockey_data['NHL W-L%'] = hockey_data['W']/ (hockey_data['W'] + hockey_data['L'] ) # getting the win-loss ratio

    keep6 =['team', 'NHL W-L%'] # choosijg specific columns to work with
    hockey_df = hockey_data[keep6]
    
    # the function below is for splitting the elements in team column, and obtaining the team name, and metropolitan area
    
    def siuu(param):
        param['NHL Team'] = param['team'].split()[-1]
        param['Metropolitan area'] = param['team'].split()[0]
        return param

    hockey_df = hockey_df.apply(siuu, 1)

    hockey_df.reset_index(drop=True, inplace=True)
    
    # renaming some metropolitan areas, and teams...
    
    hockey_df['Metropolitan area'].replace(
    {
        'Tampa': 'Tampa Bay Area',
        'Florida': 'Miami–Fort Lauderdale',
        'Washington': 'Washington, D.C.',
        'New': 'New York City',
        'Carolina': 'Raleigh',
        'New': 'New York City',
        'New': 'New York City',
        'Minnesota': 'Minneapolis–Saint Paul',
        'Colorado': 'Denver',
        'St.': 'St. Louis',
        'Dallas': 'Dallas–Fort Worth',
        'Vegas': 'Las Vegas',
        'Anaheim': 'Los Angeles',
        'San': 'San Francisco Bay Area',
        'Los': 'Los Angeles',
        'Arizona': 'Phoenix',
    }, inplace=True)
    
    hockey_df['NHL Team'].replace(
    {
        'Wings': 'Red Wings',
        'Jackets': 'Blue Jackets'
    }, inplace=True)
    
    hockey_df.drop(columns = 'team', inplace=True) # column is no longer useful
    
    keep7 = ['Metropolitan area', 'NHL Team', 'NHL W-L%'] # choosijg specific columns to work with
    
    hockey_df = hockey_df[keep7]
    return hockey_df
    
hockey = hockey() # making it a global variable

# merging hockey dataframe with 'cleaned' city dataframe  below...
def use():
    use = pd.merge(left = hockey, right= my_cities1, on='Metropolitan area', how='inner')
    use = use[['Metropolitan area', 'NHL Team', 'NHL W-L%', 'Population']]
    return use

use = use() # making it a global variable

# the function below is used to find the p_value & correlation between poplation & win-loss by region, for hockey

def nhl_correlation():
    use_agg = use.groupby('Metropolitan area')[['Population', 'NHL W-L%']].apply(np.mean)
    
    population_by_region = use_agg['Population']
    win_loss_by_region = use_agg['NHL W-L%']
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

nhl_correlation()

(0.012486162921209881, 0.9497182859911791)

In [5]:
def basketball():    
    basketball_data = pd.read_csv('assets/nba.csv')
    basketball_data.columns = [columns.strip() for columns in basketball_data.columns]
    basketball_df = basketball_data.iloc[:30, :]

    keep=['team', 'W/L%', 'year']
    basketball_df = basketball_df[keep]

    basketball_df['W/L%'] = basketball_df['W/L%'].astype(float)

    basketball_df['team'].replace(
    to_replace= '[*]?\s*\([\d]+\)', value='', regex=True, inplace=True
    )

# the function below splits the team column, and also creates a new 'Metropolitan area' column

    def jii(param):
        param['Metropolitan area'] = param['team'].split()[0]
        return param

    basketball_df = basketball_df.apply(jii, 1)

# renaming metropolitan areas...
    basketball_df['Metropolitan area'].replace(
    {
        'Miami': 'Miami–Fort Lauderdale',
        'Washington': 'Washington, D.C.',
         'Oklahoma': 'Oklahoma City',
        'San': 'San Antonio',
        'Minnesota': 'Minneapolis–Saint Paul',
        'Dallas': 'Dallas–Fort Worth',
        'Los': 'Los Angeles',
        'Indiana': 'Indianapolis',
        'Utah': 'Salt Lake City',
        'Golden': 'San Francisco Bay Area'
    }, inplace=True)
    
    basketball_df['Metropolitan area'][10] = 'New York City'
    basketball_df['Metropolitan area'][11] = 'New York City'
    basketball_df['Metropolitan area'][20] = 'New Orleans'

# the function below splits the team column, and also creates a new 'nba team' column
    def jii2(param):
        param['NBA Team'] = param['team'].split()[-1]
        return param
    
    basketball_df = basketball_df.apply(jii2, 1)
    
    basketball_df['NBA Team'][17] = 'Trail Blazers' # renamed blazers as Trail Blazers

    basketball_df.drop(columns = 'team', inplace=True) # no longer useful

    keep2 = ['Metropolitan area', 'NBA Team', 'W/L%']

    basketball_df = basketball_df[keep2]
    basketball_df['Metropolitan area'] = [ggg.strip() for ggg in basketball_df['Metropolitan area']]
    basketball_df.rename(columns = {'W/L%': 'NBA W-L%'}, inplace=True)
    return basketball_df

basketball = basketball()

## merging basketball with the city dataframe
def use2():
        use2 = pd.merge(left = basketball, right=my_cities1, on= 'Metropolitan area', how= 'inner')
        use2.drop(columns = ['NFL', 'MLB', 'NBA', 'NHL'], inplace=True)
        return use2
    
use2 = use2() # making func a global variable

# the function below is used to find the p_value & correlation between poplation & win-loss by region, for basketball...

def nba_correlation():
    use2_agg = use2.groupby('Metropolitan area')[['Population', 'NBA W-L%']].apply(np.mean)
    
    population_by_region = use2_agg['Population']
    win_loss_by_region = use2_agg['NBA W-L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    
nba_correlation()

(-0.17636350642182935, 0.36932106185547353)

In [6]:
def baseball():
    baseball_data = pd.read_csv('assets/mlb.csv')
    baseball_data = baseball_data.iloc[:30, :]
    baseball_data.rename(
    columns = {'W-L%': 'MLB W-L%'},
        inplace=True
    )

    keep8 = ['team', 'MLB W-L%']
    baseball_df = baseball_data[keep8]

    def ibra(param):
        param['MLB Team'] = param['team'].split()[-1]
        param['Metropolitan area'] = param['team'].split()[0]
        return param

    baseball_df = baseball_df.apply(ibra, 1)

# renaming some metropolitan area...
    baseball_df['Metropolitan area'].replace(
    {
        'Tampa': 'Tampa Bay Area',
        'Washington': 'Washington, D.C.',
        'New': 'New York City',
        'Carolina': 'Raleigh',
        'Minnesota': 'Minneapolis–Saint Paul',
        'Colorado': 'Denver',
        'St.': 'St. Louis',
        'Texas': 'Dallas–Fort Worth',
        'Vegas': 'Las Vegas',
        'Los': 'Los Angeles',
        'Arizona': 'Phoenix',
        'Kansas': 'Kansas City'
    }, inplace=True)  
    baseball_df['Metropolitan area'][28] = 'San Francisco Bay Area'
    baseball_df['Metropolitan area'][29] = 'San Diego'
    baseball_df['Metropolitan area'][11] = 'San Francisco Bay Area'
    baseball_df['Metropolitan area'][19]= 'Miami–Fort Lauderdale'

# renaming some mlb team...
    baseball_df['MLB Team'][0] = 'Red Sox'
    baseball_df['MLB Team'][3] = 'Blue Jays'
    baseball_df['MLB Team'][8] ='White Sox'

    baseball_df.drop(columns = 'team', inplace=True)
    keep9 = ['Metropolitan area', 'MLB Team', 'MLB W-L%']
    baseball_df = baseball_df[keep9]
    return baseball_df

baseball = baseball() # making func a global variable...

# merging basketball with the city dataframe...
def use3():
    use3 = pd.merge(left = baseball, right= my_cities1, on='Metropolitan area', how='inner')
    use3.drop(columns = ['NFL', 'MLB', 'NBA', 'NHL'], inplace=True)
    return use3

use3 = use3() # making func a global variable...

# the function below is used to find the p_value & correlation between poplation & win-loss by region, for baseball...
def mlb_correlation():
    use3_agg = use3.groupby('Metropolitan area')[['Population', 'MLB W-L%']].apply(np.mean)
    
    population_by_region = use3_agg['Population']
    win_loss_by_region = use3_agg['MLB W-L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

mlb_correlation()

(0.15003737475409498, 0.46442827201123427)

In [7]:
def football():
    football_data = pd.read_csv('assets/nfl.csv')
    football_data = football_data.iloc[:40, :]
    football_data.drop(index = [0, 5, 10, 15, 20, 25, 30, 35], inplace = True)
    football_data.reset_index(drop=True, inplace=True)

    keep3 = ['team', 'year', 'W-L%']
    football_df =football_data[keep3]

    football_df['team'].replace(
    to_replace='\*?\+?', value='', regex = True, inplace=True)


    def truu(param):
        param['NFL Team'] = param['team'].split()[-1]
        param['Metropolitan area'] = param['team'].split()[0]
        return param

    football_df = football_df.apply(truu, 1)

# renaming metropolitan areas...
    football_df['Metropolitan area'].replace(
    {
        'Tennessee': 'Nashville',
        'Oakland':  'San Francisco Bay Area',
        'Carolina': 'Charlotte',
        'Tampa': 'Tampa Bay Area',
        'San': 'San Francisco Bay Area',
        'Arizona': 'Phoenix',
        'Green': 'Green Bay',
        'Washington': 'Washington, D.C.',
        'Minnesota': 'Minneapolis–Saint Paul',
        'Dallas': 'Dallas–Fort Worth',
        'Miami': 'Miami–Fort Lauderdale',
         'Oklahoma': 'Oklahoma City',
        'Minnesota': 'Minneapolis–Saint Paul',
        'Dallas': 'Dallas–Fort Worth',
        'Los': 'Los Angeles',
        'Indiana': 'Indianapolis',
         'Kansas': 'Kansas City'
        }, inplace=True)
    
    football_df['Metropolitan area'][0] = 'Boston'
    football_df['Metropolitan area'][3] = 'New York City'
    football_df['Metropolitan area'][19] = 'New York City'
    football_df['Metropolitan area'][24] = 'New Orleans'

    football_df.drop(columns = 'team', inplace=True)

    keep4 = ['Metropolitan area', 'NFL Team', 'W-L%']
    football_df = football_df[keep4]

    football_df.rename(columns = {'W-L%': 'NFL W-L%'}, inplace=True)
    return football_df

football = football()  # making the func a global variable...

# merging basketball with the city dataframe 
def use4():
    use4 = pd.merge(left =football, right = my_cities1, on='Metropolitan area', how='inner')
    use4.drop(columns = ['NFL', 'MLB', 'NBA', 'NHL'], inplace=True)
    use4['NFL W-L%'] = use4['NFL W-L%'].astype(float)
    return use4
 
use4 = use4() # making the func a global variable...

# the function below is used to find the p_value & correlation between poplation & win-loss by region, for foottball...
def nfl_correlation():

    use4_agg = use4.groupby('Metropolitan area')[['Population', 'NFL W-L%']].apply(np.mean)

    population_by_region =use4_agg['Population']
    win_loss_by_region = use4_agg['NFL W-L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

nfl_correlation()

(0.004282141436393022, 0.9824114740736553)

__In the cell below, I would like to explore the hypothesis that given that an area has two sports teams in different sports, those teams will perform the same within their respective sports. How I would like to see this explored is with a series of paired t-tests (so use ttest_rel) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, I would find the average values where a sport has multiple teams in one region. Remember, I will only be including, for each sport, cities which have teams engaged in that sport, I will drop others as appropriate.__

In [8]:
def sports_team_performance():
    merge = basketball.merge(hockey, on='Metropolitan area')
    merge.drop(columns=['NBA Team', 'NHL Team'], inplace=True)
    merge = merge.groupby('Metropolitan area')[['NBA W-L%', 'NHL W-L%']].apply(np.mean)
    ttest, p1 = stats.ttest_ind(merge['NBA W-L%'], merge['NHL W-L%'])

    merge2 = baseball.merge(football, on='Metropolitan area')
    merge2.drop(columns=['MLB Team', 'NFL Team'], inplace=True)
    merge2['NFL W-L%'] = merge2['NFL W-L%'].astype(float)
    merge2 = merge2.groupby('Metropolitan area')[['MLB W-L%', 'NFL W-L%']].apply(np.mean)
    ttest2, p2 = stats.ttest_ind(merge2['MLB W-L%'], merge2['NFL W-L%'])

    merge3 = baseball.merge(hockey, on='Metropolitan area')
    merge3.drop(columns=['MLB Team', 'NHL Team'], inplace=True)
    merge3 = merge3.groupby('Metropolitan area')[['MLB W-L%', 'NHL W-L%']].apply(np.mean)
    ttest3, p3 = stats.ttest_ind(merge3['MLB W-L%'], merge3['NHL W-L%'])

    merge4 = baseball.merge(basketball, on='Metropolitan area')
    merge4.drop(columns=['MLB Team', 'NBA Team'], inplace=True)
    merge4 = merge4.groupby('Metropolitan area')[['MLB W-L%', 'NBA W-L%']].apply(np.mean)
    ttest4, p4 = stats.ttest_ind(merge4['MLB W-L%'], merge4['NBA W-L%'])

    merge5 = football.merge(hockey, on='Metropolitan area')
    merge5.drop(columns=['NFL Team', 'NHL Team'], inplace=True)
    merge5['NFL W-L%'] = merge5['NFL W-L%'].astype(float)
    merge5 = merge5.groupby('Metropolitan area')[['NFL W-L%', 'NHL W-L%']].apply(np.mean)
    ttest5, p5 = stats.ttest_ind(merge5['NFL W-L%'], merge5['NHL W-L%'])

    merge6 = football.merge(basketball, on='Metropolitan area')
    merge6.drop(columns=['NBA Team', 'NFL Team'], inplace=True)
    merge6['NFL W-L%'] = merge6['NFL W-L%'].astype(float)
    merge6 = merge6.groupby('Metropolitan area')[['NFL W-L%', 'NBA W-L%']].apply(np.mean)
    ttest6, p6 = stats.ttest_ind(merge6['NFL W-L%'], merge6['NBA W-L%'])

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame(
    {'NFL': [np.NaN, p6, p5, p2],
     'NBA': [p6, np.NaN, p1, p4],
     'NHL': [p5, p1, np.NaN, p3],
     'MLB': [p2, p4, p3, np.NaN]
    }, 
        index=sports
    )   
   
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.944874,0.051097,0.798408
NBA,0.944874,NaN,0.144596,0.955988
NHL,0.051097,0.144596,NaN,0.003222
MLB,0.798408,0.955988,0.003222,NaN
